# Semantic Kernel na Muunganisho wa OpenBnB MCP Server

Noti hii inaonyesha jinsi ya kutumia Semantic Kernel na seva halisi ya OpenBnB MCP kutafuta makazi halisi ya Airbnb kwa kutumia MCPStdioPlugin. Kwa ufikiaji wa LLM, inatumia Azure AI Foundry. Ili kusanidi vigezo vya mazingira yako, unaweza kufuata [Somo la Usanidi](/00-course-setup/README.md)


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio

from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## Kuunda Muunganisho wa Kiongezi cha MCP

Tutaunganishwa na [seva ya OpenBnB MCP](https://github.com/openbnb-org/mcp-server-airbnb) tukitumia MCPStdioPlugin. Seva hii inatoa uwezo wa kutafuta Airbnb kupitia kifurushi cha @openbnb/mcp-server-airbnb.


## Kuunda Mteja

Katika mfano huu, tutatumia Azure AI Foundry kwa upatikanaji wetu wa LLM. Hakikisha kwamba vigezo vya mazingira vimewekwa vizuri.


## Usanidi wa Mazingira

Sanidi mipangilio ya Azure OpenAI. Hakikisha una vigezo vya mazingira vifuatavyo vimewekwa:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## Kuelewa Muunganisho wa OpenBnB MCP

Notebook hii inaunganishwa na **seva halisi ya OpenBnB MCP** inayotoa uwezo halisi wa kutafuta Airbnb.

### Jinsi inavyofanya kazi:

1. **MCPStdioPlugin**: Inatumia mawasiliano ya pembejeo/pato la kawaida na seva ya MCP
2. **Kifurushi Halisi cha NPM**: Inapakua na kuendesha `@openbnb/mcp-server-airbnb` kupitia npx
3. **Takwimu za Moja kwa Moja**: Inarejesha data halisi ya mali za Airbnb kutoka kwa API zao
4. **Ugunduzi wa Kazi**: Wakala hugundua kiotomatiki kazi zinazopatikana kutoka kwa seva ya MCP

### Kazi Zinazopatikana:

Seva ya OpenBnB MCP kwa kawaida hutoa:
- **search_listings** - Tafuta mali za Airbnb kwa eneo na vigezo
- **get_listing_details** - Pata maelezo ya kina kuhusu mali maalum
- **check_availability** - Angalia upatikanaji kwa tarehe maalum
- **get_reviews** - Pata hakiki za mali
- **get_host_info** - Pata maelezo kuhusu wenyeji wa mali

### Mahitaji:

- **Node.js** imewekwa kwenye mfumo wako
- **Muunganisho wa Intaneti** kupakua kifurushi cha seva ya MCP
- **NPX** kinapatikana (kinakuja na Node.js)

### Kupima Muunganisho:

Unaweza kupima seva ya MCP kwa mikono kwa kuendesha:
```bash
npx -y @openbnb/mcp-server-airbnb
```

Hii itapakua na kuanzisha seva ya OpenBnB MCP, ambayo Semantic Kernel kisha inaunganishwa nayo kwa data halisi ya Airbnb.


## Kuendesha Wakala na OpenBnB MCP Server

Sasa tutaendesha Wakala wa AI ambaye anaunganishwa na OpenBnB MCP server kutafuta malazi halisi ya Airbnb huko Stockholm kwa watu wazima 2 na mtoto 1. Unaweza kubadilisha orodha ya `user_inputs` ili kurekebisha vigezo vya utafutaji.


In [ ]:
# Main execution cell - Enhanced with proper HTML rendering and MCP tool logging
# User requests for Airbnb search
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        # Create MCP plugin connection to real OpenBnB server
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb", "--ignore-robots-txt"],
        ) as airbnb_plugin:

            print("🔧 MCP Plugin created and connected")

            # Load tools for function discovery
            await airbnb_plugin.load_tools()
            await asyncio.sleep(3)  # Give more time for initialization
            print("✅ Tools loaded from MCP server")

            # Debug: Check what tools were loaded
            if hasattr(airbnb_plugin, '_tools'):
                print(f"📋 Internal tools: {airbnb_plugin._tools}")

            # Verify available functions
            funcs = [attr for attr in dir(airbnb_plugin)
                     if callable(getattr(airbnb_plugin, attr))
                     and attr in ['airbnb_search', 'airbnb_listing_details']]
            print(f"📋 Available functions: {funcs}")

            # Create agent with Azure OpenAI service
            agent = ChatCompletionAgent(
                service=AzureChatCompletion(),  # Use default constructor
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the airbnb_search function to find properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("🤖 Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 Processing request: {user_input}")
                
                # Track MCP tool usage
                mcp_tools_used = []
                function_calls_log = []
                
                # Try streaming to capture function calls
                try:
                    agent_name = None
                    full_response = []
                    current_function_name = None
                    argument_buffer = ""
                    
                    async for response in agent.invoke_stream(
                        messages=user_input,
                        thread=thread,
                    ):
                        thread = response.thread
                        agent_name = response.name
                        
                        for item in response.items:
                            # Log function calls
                            if isinstance(item, FunctionCallContent):
                                if item.function_name:
                                    current_function_name = item.function_name
                                    mcp_tools_used.append(item.function_name)
                                    print(f"\n🔧 MCP Tool Selected: {item.function_name}")
                                    
                                if isinstance(item.arguments, str):
                                    argument_buffer += item.arguments
                            
                            # Log function results
                            elif isinstance(item, FunctionResultContent):
                                if current_function_name:
                                    try:
                                        args = json.loads(argument_buffer.strip()) if argument_buffer else {}
                                    except:
                                        args = {"raw": argument_buffer}
                                    
                                    function_calls_log.append({
                                        "function": current_function_name,
                                        "arguments": args,
                                        "timestamp": asyncio.get_event_loop().time()
                                    })
                                    
                                    print(f"   📍 Arguments: {json.dumps(args, indent=2)}")
                                    print(f"   ✅ MCP Tool Executed Successfully")
                                    
                                    current_function_name = None
                                    argument_buffer = ""
                            
                            # Collect response text
                            elif isinstance(item, StreamingTextContent) and item.text:
                                full_response.append(item.text)
                    
                    # Join the full response
                    response_text = ''.join(full_response)
                    
                except Exception as e:
                    print(f"⚠️ Streaming failed, using get_response: {str(e)[:100]}")
                    # Fallback to non-streaming
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    response_text = str(response)
                    agent_name = response.name
                
                
                # Process the response to ensure HTML tables render correctly
                # Remove any markdown code blocks around HTML
                response_text = response_text.replace('```html', '').replace('```', '')
                
                # Ensure proper HTML structure for tables
                if '<table' in response_text.lower():
                    # Add CSS styling for better table rendering
                    table_css = """
                    <style>
                        .airbnb-results table {
                            border-collapse: collapse;
                            width: 100%;
                            margin: 10px 0;
                        }
                        .airbnb-results th, .airbnb-results td {
                            border: 1px solid #ddd;
                            padding: 8px;
                            text-align: left;
                        }
                        .airbnb-results th {
                            background-color: #f2f2f2;
                            font-weight: bold;
                        }
                        .airbnb-results tr:nth-child(even) {
                            background-color: #f9f9f9;
                        }
                        .airbnb-results a {
                            color: #1976d2;
                            text-decoration: none;
                        }
                        .airbnb-results a:hover {
                            text-decoration: underline;
                        }
                    </style>
                    """
                    response_text = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                
                # Build the complete HTML output
                html_output = f"""
                <div style='margin:10px; padding:10px; border-left:3px solid #2E8B57; background:#F0F8FF;'>
                    <strong>User:</strong> {user_input}
                </div>
                """
                
                # Add function call details if available
                if function_calls_log:
                    details_html = "<details style='margin:10px; padding:10px; background:#f5f5f5;'>"
                    details_html += "<summary><strong>📊 Function Call Details</strong></summary>"
                    details_html += "<pre style='background:#fff; padding:10px; overflow-x:auto;'>"
                    for call in function_calls_log:
                        details_html += f"Function: {call['function']}\n"
                        details_html += f"Arguments: {json.dumps(call['arguments'], indent=2)}\n"
                        details_html += "---\n"
                    details_html += "</pre></details>"
                    html_output += details_html
                
                # Add the agent's response with proper HTML rendering
                html_output += f"""
                <div style='margin:10px; padding:15px; border-left:3px solid #1E90FF; background:#FFFFFF;'>
                    <strong>{agent_name}:</strong><br>
                    {response_text}
                </div>
                """
                
                # Display the HTML with proper rendering
                display(HTML(html_output))
                
                
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        import traceback
        traceback.print_exc()

print("🚀 Starting with Azure OpenAI...")
await main()
print("✅ Done!")

Muhtasari  
Hongera! Umefanikiwa kujenga wakala wa AI unaounganisha na utafutaji wa malazi halisi kwa kutumia Model Context Protocol (MCP):  

Teknolojia Zilizotumika:  
- Semantic Kernel - Kwa kujenga mawakala wenye akili kwa Azure OpenAI  
- Azure AI Foundry - Kwa uwezo wa LLM na kukamilisha mazungumzo  
- MCP (Model Context Protocol) - Kwa ujumuishaji wa zana uliosanifishwa  
- OpenBnB MCP Server - Kwa utafutaji halisi wa Airbnb  
- Node.js/NPX - Kwa kuendesha seva ya nje ya MCP  

Ulichopata:  
- Ujumuishaji wa MCP: Kuunganisha mawakala wa Semantic Kernel na seva za nje za MCP  
- Ufikiaji wa Data ya Wakati Halisi: Kutafuta mali halisi za Airbnb kupitia API za moja kwa moja  
- Mawasiliano ya Itifaki: Kutumia mawasiliano ya stdio kati ya wakala na seva ya MCP  
- Ugunduzi wa Kazi: Kugundua kiotomatiki kazi zinazopatikana kutoka kwa seva za MCP  
- Majibu ya Mfululizo: Kukamata na kurekodi miito ya kazi kwa wakati halisi  
- Utoaji wa HTML: Kuweka majibu ya wakala kwa meza zenye mitindo na maonyesho yanayoweza kuingiliana  

Hatua Zifuatazo:  
- Jumuisha seva za ziada za MCP (hali ya hewa, ndege, migahawa)  
- Jenga mfumo wa mawakala wengi unaochanganya MCP na itifaki za A2A  
- Unda seva za MCP maalum kwa vyanzo vyako vya data  
- Tekeleza kumbukumbu ya mazungumzo ya kudumu katika vipindi vyote  
- Peleka wakala kwa Azure Functions na uratibu wa seva ya MCP  
- Ongeza uthibitishaji wa mtumiaji na uwezo wa kuhifadhi  

Faida Muhimu za Usanifu wa MCP:  
- Usanifishaji: Itifaki ya ulimwengu kwa kuunganisha mawakala wa AI na zana za nje  
- Data ya Wakati Halisi: Ufikiaji wa habari ya moja kwa moja, ya kisasa kutoka kwa huduma mbalimbali  
- Uwezo wa Kupanuka: Ujumuishaji rahisi wa vyanzo vipya vya data na zana  
- Uingiliano: Inafanya kazi katika mifumo tofauti ya AI na majukwaa ya mawakala  
- Utengano wa Majukumu: Tofauti wazi kati ya mantiki ya AI na ufikiaji wa data ya nje  



---

**Kanusho**:  
Hati hii imetafsiriwa kwa kutumia huduma ya kutafsiri ya AI [Co-op Translator](https://github.com/Azure/co-op-translator). Ingawa tunajitahidi kuhakikisha usahihi, tafadhali fahamu kuwa tafsiri za kiotomatiki zinaweza kuwa na makosa au kutokuwa sahihi. Hati ya asili katika lugha yake ya awali inapaswa kuzingatiwa kama chanzo cha mamlaka. Kwa taarifa muhimu, tafsiri ya kitaalamu ya binadamu inapendekezwa. Hatutawajibika kwa kutoelewana au tafsiri zisizo sahihi zinazotokana na matumizi ya tafsiri hii.
